In [215]:
import re
import sys
import requests
import os


In [229]:
apps = ['account', 'my_health_info', 'exercises_info', 'community', 'etc']

env = os.environ
pat = env.get('PAT')

owner = "orm-backend-final-project-2-2"
repo = "final-project-server"
token = pat

project_id = 2

request_headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github.v3+json"
}

asignee_map = {
    'account': 'neysin',
    'my_health_info': 'ZeyaKim',
    'exercises_info': 'vin00',
    'community': 'soohyun020812'
}

docs_path = f'docs/requirements/{app_name}_requirements.md'

In [217]:
def create_markdown_table(req_info_list, req_attributes):
    # 테이블 헤더 생성
    headers = "| " + " | ".join(req_attributes) + " |"
    # 구분선 생성
    separator = "|-" + "-|".join(["-" * len(attr) for attr in req_attributes]) + "-|"
    # 테이블 본문 생성
    rows = []
    for req_info in req_info_list:
        row = "| " + " | ".join([req_info[attr] for attr in req_attributes]) + " |"
        rows.append(row)
    # 전체 테이블 합치기
    table = "\n".join([headers, separator] + rows)
    return table

In [218]:
def get_requirement_info(line, req_attributes):
    """line에서 requirement 정보를 추출해서 dict로 반환한다."""

    req_info = dict()
    refined_line = [item.strip() for item in line.split("|")[1:-1]]
    for i, attr in enumerate(req_attributes):
        req_info[attr] = refined_line[i]

    return req_info

In [219]:
def find_issue_by_label(label):
    query = f"label:{label}+repo:{owner}/{repo}+state:open"
    url = f"https://api.github.com/search/issues?q={query}"

    response = requests.get(url, headers=request_headers)
    
    if response.status_code == 200:
        count = response.json()['total_count']
        print (f"Found {count} issues with label '{label}'")
        if count == 1:
            return response.json()['items'][0]['number']
    else:
        return None

In [220]:
def create_label(label):
    url = f"https://api.github.com/repos/{owner}/{repo}/labels"
    data = {
        "name": label,
    }
    
    print(data)
    
    response = requests.post(url, headers=request_headers, json=data)
    print(response.status_code)
    if response.status_code == 201:
        return label
    else:
        return None

def get_or_create_label(label):
    url = f"https://api.github.com/repos/{owner}/{repo}/labels"
    data = {
        "name": label,
    }
    
    response = requests.get(url, headers=request_headers, json=data)
    
    if response.status_code == 200:
        labels = response.json()  # 레이블 리스트 직접 접근
        for l in labels:
            if l["name"] == label:
                print(f"Label '{label}' already exists.")
                return label

    return create_label(label)
    
def create_milestone(milestone):
    url = f"https://api.github.com/repos/{owner}/{repo}/milestones"
    data = {
        "title": milestone
    }
    
    response = requests.post(url, headers=request_headers, json=data)
    
    if response.status_code == 201:
        print (f"Milestone '{milestone}' created successfully.")
        return response.json()["number"]
    else:
        print (f"Failed to create milestone '{milestone}'")
        return None
    
def get_or_create_milestone(milestone):
    search_url = f"https://api.github.com/repos/{owner}/{repo}/milestones"
    
    response = requests.get(search_url, headers=request_headers)
    
    if response.status_code == 200:
        milestones = response.json()

        for m in milestones:
            if m["title"] == milestone:
                print (f"Milestone '{milestone}' already exists.")
                return m["number"]
    
    return create_milestone(milestone)

In [221]:
def update_issue_by_id(issue_id, body):
    url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_id}"
    
    issue = {
        "body": body
    }
    
    response = requests.patch(url, headers=request_headers, json=issue)
    
    if response.status_code == 200:
        return f"Successfully updated an issue: {issue_id}"
    else:
        return None
    
def create_issue(app_name, title, label, milestone, body):
    url = f"https://api.github.com/repos/{owner}/{repo}/issues"

    milestone = get_or_create_milestone(milestone)
    label = get_or_create_label(label)

    issue = {
        "title": title,
        "body": body,
        "assignees": [asignee_map[app_name]],
        "milestone": milestone,
        "labels": [label, 'requirement']
    }
    
    print(issue)
    
    response = requests.post(url, headers=request_headers, json=issue)
    if response.status_code == 201:
        print(f"Successfully created an issue: {title}")
    else:
        print(f"Failed to create an issue: {response.status_code}")
        return None

In [222]:
def update_issue(app_name, issue_title, label, milestone_name, issue_body):
    issue_id = find_issue_by_label(label)
    if issue_id:
        print(f"Found an issue: {issue_id}")
        update_issue_by_id(issue_id, issue_body)
    else:
        print("No issue found.")
        create_issue(app_name, issue_title, label, milestone_name, issue_body)

In [223]:
def update_app_requirements(app_name, req_file):
    """req_file을 읽어서 정보를 map에 저장하고 app_name에 해당하는 issue를 찾아서 업데이트한다."""

    with open(req_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
        
        req_attributes = [attribute.strip() for attribute in lines[2].split("|")[1:-1]]

        app_requirements_info = dict()

        for line in lines[4:]:
            if line.startswith("|"):
                req_info = get_requirement_info(line, req_attributes)
                if req_info["Feature"] != "":
                    current_feature = req_info["Feature"]
                    app_requirements_info[current_feature] = []
                app_requirements_info[current_feature].append(req_info)

        for feature in app_requirements_info:
            issue_title = f"REQ-{app_name}-{feature}"
            label = f"{app_name}-{feature}"
            milestone_name = f"{app_name}-requirements"
            issue_body = create_markdown_table(
                app_requirements_info[feature], req_attributes
            )

            update_issue(app_name, issue_title, label, milestone_name, issue_body)

In [232]:
app_name = 'account'
req_file = f"docs/requirements/{app_name}_requirements.md"
update_app_requirements(app_name, req_file)

In [ ]:
app_name = 'exercises_info'
req_file = f"docs/requirements/{app_name}_requirements.md"
update_app_requirements(app_name, req_file)

In [ ]:
app_name = 'my_health_info'
req_file = f"docs/requirements/{app_name}_requirements.md"
update_app_requirements(app_name, req_file)

In [ ]:
app_name = 'community'
req_file = f"docs/requirements/{app_name}_requirements.md"
update_app_requirements(app_name, req_file)